In [1]:
"""
This is THE model.

It combines improvements from the LLAMA3 series, with others from Gemma2.
It has:
- GQA
- Sliding window attention every two layers
- GeGLU (thinking to maybe use SwiGLU or ReGLU)
- RoPE

I still would need to implement KV-caching to improve inference type.
"""
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import inspect

def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    """torch.repeat_interleave(x, dim=2, repeats=n_rep)"""
    bs, slen, n_kv_heads, head_dim = x.shape
    if n_rep == 1:
        return x
    return (
        x[:, :, :, None, :]
        .expand(bs, slen, n_kv_heads, n_rep, head_dim)
        .reshape(bs, slen, n_kv_heads * n_rep, head_dim)
    )


class CausalSelfAttentionGQA(nn.Module):
    def __init__(self,config):
        super().__init__()
        assert config.n_embd % config.n_head == 0

        self.head_dim = config.n_embd // config.n_head


        shape = (config.n_head + 2 * config.n_kv_heads) * self.head_dim

        self.c_attn = nn.Linear(config.n_embd, shape, bias=False) # q_proj, k_proj i v_proj joined
        self.o_proj = nn.Linear(self.head_dim * config.n_head, config.n_embd, bias=False) # o_proj
        
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        

        self.n_kv_heads = config.n_kv_heads # Nombre de grups de query

        assert self.n_head % self.n_kv_heads == 0, "n_head must be divisible by n_group"
        self.queries_per_kv = self.n_head // self.n_kv_heads # n_rep

        #self.sliding_window_size = config.sliding_window_size
        self.max_seq_len = config.block_size

    def forward(self, x, freqs_cis):
        B, T, C = x.size()

        qkv = self.c_attn(x)
        total_qkv = self.queries_per_kv + 2 # cada grup té a més de queries, 1 key i 1 value
        qkv = qkv.view(B, T, self.n_kv_heads, total_qkv, self.head_dim)
        qkv = qkv.permute(0, 2, 3, 1, 4)

        q, k, v = qkv.split((self.queries_per_kv, 1, 1), dim=2)

        q = q.reshape(B, -1, self.n_head, self.head_dim)  # (B, T, n_h, hs)
        k = k.reshape(B, -1, self.n_kv_heads, self.head_dim)  # (B, T, nh_kv, hs)
        v = v.reshape(B, -1, self.n_kv_heads, self.head_dim)

        q = apply_rope(q, freqs_cis)
        k = apply_rope(k, freqs_cis)

        if self.n_kv_heads != self.n_head:
            k = repeat_kv(k, n_rep=self.queries_per_kv)
            v = repeat_kv(v, n_rep=self.queries_per_kv)

        # (B, n_h, T, hs)
        q = q.transpose(1,2)
        k = k.transpose(1,2)
        v = v.transpose(1,2)

        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)

        """
            SLIDING WINDOW DISCARDED (FOR NOW) TO MATCH EXACTLY LLAMAFORCAUSALLM
            scores = torch.matmul(q, k.transpose(2, 3)) / math.sqrt(self.head_dim)

            all_ones = torch.ones((T, T), device=q.device)
            sliding_mask = torch.triu(all_ones, -self.sliding_window_size + 1) * torch.tril(all_ones, self.sliding_window_size - 1)
            sliding_mask = sliding_mask.unsqueeze(0).unsqueeze(0)
            mask = torch.where(sliding_mask == 1, torch.zeros_like(scores), torch.full_like(scores, float("-inf")))
            scores = scores + mask
            scores = F.softmax(scores.float(), dim=-1).type_as(q)
            # [B, n_h, T, hs]
            y = torch.matmul(scores, v)
        """

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.o_proj(y)
        return y


class RMSNorm(nn.Module):
    def __init__(self, d, eps=1e-6):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(d)) # weight

    def forward(self, x):
        norm = torch.sqrt(x.float().pow(2).mean(-1, keepdim=True) + self.eps).type_as(x)
        return self.scale.to(x.device) * (x / norm)
    

def rope_scaling(freqs: torch.Tensor):
    """
    From https://github.com/karpathy/nano-llama31/blob/master/llama31.py
    """
    scale_factor = 8
    low_freq_factor = 1
    high_freq_factor = 4
    old_context_len = 2048
    low_freq_wavelen = old_context_len / low_freq_factor
    high_freq_wavelen = old_context_len / high_freq_factor
    new_freqs = []
    for freq in freqs:
        wavelen = 2 * math.pi / freq
        if wavelen < high_freq_wavelen:
            new_freqs.append(freq)
        elif wavelen > low_freq_wavelen:
            new_freqs.append(freq / scale_factor)
        else:
            assert low_freq_wavelen != high_freq_wavelen
            smooth = (old_context_len / wavelen - low_freq_factor) / (
                high_freq_factor - low_freq_factor
            )
            new_freqs.append((1 - smooth) * freq / scale_factor + smooth * freq)
    return torch.tensor(new_freqs, dtype=freqs.dtype, device=freqs.device)

def precompute_rope(dim, end, theta, use_scaled):
    inv_freq = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim)) # theta
    
    if use_scaled:
        inv_freq = rope_scaling(inv_freq)

    position_ids = torch.arange(end, device=inv_freq.device, dtype=torch.float32) # en alguns llocs, seq_idx
    
    freqs = torch.outer(position_ids, inv_freq)
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    freqs_cis_real = torch.stack([freqs_cis.real, freqs_cis.imag], dim=-1)
    return freqs_cis_real

def apply_rope(x, freqs_cis):
    """
    Alt: Gemma implementation:
    def apply_rotary_emb(x: torch.Tensor, freqs_cis: torch.Tensor) -> torch.Tensor:
    x_ = torch.view_as_complex(
        torch.stack(torch.chunk(x.transpose(1, 2).float(), 2, dim=-1),
                    dim=-1))
    x_out = torch.view_as_real(x_ * freqs_cis).type_as(x)
    x_out = torch.cat(torch.chunk(x_out, 2, dim=-1), dim=-2)
    x_out = x_out.reshape(x_out.shape[0], x_out.shape[1], x_out.shape[2],
                          -1).transpose(1, 2)
    return x_out

    
    """


    xshaped = x.float().reshape(*x.shape[:-1], -1, 2)
    # xshaped is (bs, seqlen, n_heads, head_dim/2, 2), e.g. (4, 8, 32, 64, 2)

    freqs_cis = freqs_cis.view(1, xshaped.size(1), 1, xshaped.size(3), 2)
    # freqs_cis becomes (1, seqlen, 1, head_dim/2, 2), e.g. (1, 8, 1, 64, 2)
    
    x_out2 = torch.stack(
        [
            xshaped[..., 0] * freqs_cis[..., 0] - xshaped[..., 1] * freqs_cis[..., 1],
            xshaped[..., 1] * freqs_cis[..., 0] + xshaped[..., 0] * freqs_cis[..., 1],
        ],
        -1,
    )
    # x_out2 at this point is (bs, seqlen, n_heads, head_dim/2, 2), e.g. (4, 8, 32, 64, 2)
    x_out2 = x_out2.flatten(3)
    # x_out2 is now (bs, seqlen, n_heads, head_dim), e.g. (4, 8, 32, 128)

    return x_out2.type_as(x)


class MLP(nn.Module):
    """
    class GeGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1)
        # Silu és el mateix que swift function
        return F.gelu(gate) * x
    
    """
    def __init__(self, config):
        super().__init__()
        self.w1   = nn.Linear(config.n_embd, config.intermediate_size, bias=False) # gate_proj
        self.w2 = nn.Linear(config.intermediate_size, config.n_embd, bias=False) # down proj
        self.w3 = nn.Linear(config.n_embd, config.intermediate_size, bias=False) # up proj
    
    def forward(self, x):
        return self.w2(F.gelu(self.w1(x), approximate="tanh")* self.w3(x)) 


class Block(nn.Module):

    def __init__(self,config):
        super().__init__()
        self.norm_1 = RMSNorm(config.n_embd, config.norm_eps) # input_layernorm
        self.attn = CausalSelfAttentionGQA(config)
        self.norm_2 = RMSNorm(config.n_embd, config.norm_eps) # post_attention_layernorm
        self.mlp = MLP(config)
    
    def forward(self, x, freq):
        x = x + self.attn(self.norm_1(x), freq)
        x = x + self.mlp(self.norm_2(x))
        return x

class Aloja(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd, padding_idx=3), #embed_tokens.weight
            h = nn.ModuleList([Block(config) for i in range(config.n_layer)]),
            norm_f =  RMSNorm(config.n_embd, config.norm_eps),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.transformer.wte.weight = self.lm_head.weight # Linkeddddd

        self.freqs = precompute_rope(config.n_embd // config.n_head, config.max_seq_len * 2, config.rope_theta, config.use_scaled_rope)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            torch.nn.init.normal_(module.weight, mean = 0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, start_pos:int = 0):
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size is smaller"

        x = self.transformer.wte(idx)
        self.freqs = self.freqs.to(x.device)
        freqs = self.freqs[:T]
        mask = torch.full((T, T), float("-inf"), device=idx.device)
        mask = torch.triu(mask, diagonal=1)
        mask = mask.type_as(x)

        for block in self.transformer.h:
            x = block(x, freqs, mask)
        x = self.transformer.norm_f(x)
        logits = self.lm_head(x).float()
        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                input=logits.transpose(1, 2),
                target=targets,
                reduction="mean",
                ignore_index=3,
            )
        return logits, loss
    
    
    def configure_optimizers(self, learning_rate, weight_decay=0.0, betas=(0.9, 0.97), device_type='cuda'):
        # start with all of the candidate parameters (that require grad)
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        if master_process:
            print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
            print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == "cuda"
        if master_process:
            print(f"using fused AdamW: {use_fused}")
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, eps=1e-8, fused=use_fused)
        return optimizer


@dataclass
class AlojaConfig:
    block_size: int = 2048
    vocab_size: int = 65536
    n_layer: int = 32
    n_head: int = 8
    n_embd: int = 768
    intermediate_size = 2048 # 3072
    n_kv_heads: int = 4 # nombre de grups de query
    norm_eps: int = 1e-5
    rope_theta: float = 500000
    use_scaled_rope: bool = False
    max_batch_size: int = 32
    max_seq_len:int = 2048


In [ ]:
def load_checkpoint(model, optimizer, dataloader, checkpoint_path, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.config(checkpoint['config'])
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    step = checkpoint['step']
    val_loss = checkpoint['val_loss']
    print(f"Checkpoint loaded from step {step} with val loss {val_loss}")
    return step


In [5]:
checkpoint = torch.load("./src/model_00576.pt", map_location="cpu")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_19512\3942902272.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("./src/model_00576.pt", map_locati

In [7]:
model = Aloja(checkpoint['config'])

In [8]:
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
def visualize_attention(model, tokenizer, sentence, layer_num, head_num):
    # Tokenize the input sentence
    tokens = tokenizer(sentence, return_tensors="pt")
    input_ids = tokens["input_ids"]

    # Move input to the same device as the model
    input_ids = input_ids.to(next(model.parameters()).device)

    # Forward pass through the model with return_attention=True
    _, attention_weights = model.transformer.h[layer_num].attn(model.transformer.wte(input_ids), model.freqs, return_attention=True)

    # Select the attention weights for the specified head
    attention_weights = attention_weights[0, head_num].detach().cpu().numpy()

    # Plot the attention weights
    plt.figure(figsize=(10, 8))
    sns.heatmap(attention_weights, cmap="viridis", xticklabels=sentence.split(), yticklabels=sentence.split())
    plt.title(f"Attention Weights - Layer {layer_num + 1}, Head {head_num + 1}")
    plt.xlabel("Key Position")
    plt.ylabel("Query Position")
    plt.show()

In [ ]:
visualize_attention(model, enc, )

[0.012831157073378563,
 0.0065925405360758305,
 -0.007648141589015722,
 0.019636638462543488,
 0.02578912302851677,
 0.015744853764772415,
 -0.023119593039155006,
 -0.01135143730789423,
 -0.011666991747915745,
 0.019317064434289932,
 0.020809516310691833,
 0.00041685145697556436,
 -0.010931160300970078,
 0.015505609102547169,
 -0.008102518506348133,
 0.019798098132014275,
 0.06340011954307556,
 -0.009394454769790173,
 0.005274095572531223,
 -0.015478570014238358,
 0.007816036231815815,
 -0.012796544469892979,
 0.024425633251667023,
 -0.013300667516887188,
 -0.016700919717550278,
 0.014346715994179249,
 -0.006231224630028009,
 0.00986629631370306,
 -0.004729934502393007,
 0.015897955745458603,
 -0.000613517127931118,
 -0.010332741774618626,
 0.024980813264846802,
 -0.010868091136217117,
 -0.0029572120402008295,
 0.016337065026164055,
 0.004842178430408239,
 -0.004348214250057936,
 0.013645288534462452,
 -0.0030171643011271954,
 -0.009156519547104836,
 -0.016477586701512337,
 0.009412320

In [12]:
list(model.state_dict())[:4]

['transformer.wte.weight',
 'transformer.h.0.norm_1.scale',
 'transformer.h.0.attn.c_attn.weight',
 'transformer.h.0.attn.o_proj.weight']

In [21]:
plt.figure(figsize=(10, 8))
sns.heatmap(model.state_dict()["transformer.h.0.attn.c_attn.weight"][0].tolist(), cmap="viridis")
plt.title(f"Attention Weights")
plt.xlabel("Key Position")
plt.ylabel("Query Position")
plt.show()

IndexError: Inconsistent shape between the condition and the input (got (768, 1) and (768,))

<Figure size 1000x800 with 0 Axes>

tensor([ 1.2831e-02,  6.5925e-03, -7.6481e-03,  1.9637e-02,  2.5789e-02,
         1.5745e-02, -2.3120e-02, -1.1351e-02, -1.1667e-02,  1.9317e-02,
         2.0810e-02,  4.1685e-04, -1.0931e-02,  1.5506e-02, -8.1025e-03,
         1.9798e-02,  6.3400e-02, -9.3945e-03,  5.2741e-03, -1.5479e-02,
         7.8160e-03, -1.2797e-02,  2.4426e-02, -1.3301e-02, -1.6701e-02,
         1.4347e-02, -6.2312e-03,  9.8663e-03, -4.7299e-03,  1.5898e-02,
        -6.1352e-04, -1.0333e-02,  2.4981e-02, -1.0868e-02, -2.9572e-03,
         1.6337e-02,  4.8422e-03, -4.3482e-03,  1.3645e-02, -3.0172e-03,
        -9.1565e-03, -1.6478e-02,  9.4123e-03, -7.9972e-04, -1.8653e-02,
         4.6008e-03, -1.0532e-02, -1.0414e-02,  7.0509e-03, -5.6482e-03,
         4.0825e-03, -1.6808e-02, -4.9664e-04, -4.4581e-03,  1.6152e-03,
         6.4731e-04,  6.1736e-03,  4.1046e-03,  2.2668e-02,  6.3569e-03,
         4.0307e-02,  5.8478e-03, -3.0477e-03,  6.7836e-03,  1.6529e-02,
         2.4309e-02,  1.7485e-02,  1.1264e-02,  6.6

In [ ]:
load_checkpoint()